In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [13]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(name="ai-professor", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [ ]:
import json
data = json.load(open("reviews.json"))
data['reviews']

In [9]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['comment'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": str(review['id']),
        "metadata": {
            "professor_name": review['professor_name'],
            "department": review['department'],
            "course_code": review['course_code'],
            "rating": review['rating'],
            "difficulty": review['difficulty'],
            "comment": review['comment'],
            "would_take_again": review['would_take_again'],
            "date": review['date'],
            "grade": review["grade"]
        }
    })

In [ ]:
processed_data[0]

In [ ]:
index = pc.Index("ai-professor")
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

In [ ]:
index.describe_index_stats()